<a href="https://colab.research.google.com/github/lucasestrela/Dissertacao/blob/main/Construindo_DataFrame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Importando pacotes

In [1]:
pip install wbgapi

In [2]:
import numpy as np
import pandas as pd
import wbgapi as wb
from google.colab import files

In [3]:
#############################################################################
######################## --- Lendo Base de Dados --- ########################
#############################################################################

cbi = pd.read_excel('https://github.com/lucasestrela/Dissertacao/blob/main/Dados/CBI_data_2019.xlsx?raw=true',
                    sheet_name = 'Sheet1'
                    )



fr = pd.read_stata('https://github.com/lucasestrela/Dissertacao/blob/main/Dados/fiscal_rule_database.dta?raw=true'
                    )



ipd = pd.read_excel('https://github.com/lucasestrela/Dissertacao/blob/main/Dados/IPD_2016.xlsx?raw=true',
                    sheet_name = 'Variables',
                    header = 0
                    )


pwt = pd.read_excel('https://github.com/lucasestrela/Dissertacao/blob/main/Dados/pwt100.xlsx?raw=true',
                    sheet_name = 'Data'
                    )



*   Unemployment, total (% of total labor force)  

https://data.worldbank.org/indicator/SL.UEM.TOTL.ZS?view=chart

*   Income share held by lowest 10%

https://data.worldbank.org/indicator/SI.DST.FRST.10?view=chart

*   Social contributions (% of revenue)

https://data.worldbank.org/indicator/GC.REV.SOCL.ZS

*   Poverty headcount ratio at $1.90 a day (2011 PPP) (% of population)

https://data.worldbank.org/indicator/SI.POV.DDAY?view=chart

*   Population ages 65 and above (% of total population)

https://data.worldbank.org/indicator/SP.POP.65UP.TO.ZS?view=chart

*   Population ages 0-14 (% of total population)

https://data.worldbank.org/indicator/SP.POP.0014.TO.ZS?view=chart

*   Age dependency ratio (% of working-age population)

https://data.worldbank.org/indicator/SP.POP.DPND?view=chart

*   Prevalence of undernourishment (% of population)

https://data.worldbank.org/indicator/SN.ITK.DEFC.ZS

*   Inflation, consumer prices (annual %)

https://data.worldbank.org/indicator/FP.CPI.TOTL.ZG?view=chart

*   Strength of legal rights index

https://data.worldbank.org/indicator/IC.LGL.CRED.XQ?view=chart

*   Government expenditure on education, total (% of GDP)

https://data.worldbank.org/indicator/SE.XPD.TOTL.GD.ZS?view=chart


*   Gross savings (% of GDP)

https://data.worldbank.org/indicator/NY.GNS.ICTR.ZS?view=chart

*   GDP growth (annual %)

https://data.worldbank.org/indicator/NY.GDP.MKTP.KD.ZG?view=chart

---

https://data.worldbank.org/indicator?tab=featured



In [ ]:
wb_cod = ['NY.GDP.MKTP.KD.ZG',
          'NY.GNS.ICTR.ZS',
          'SE.XPD.TOTL.GD.ZS',
          'IC.LGL.CRED.XQ',
          'FP.CPI.TOTL.ZG',
          'SN.ITK.DEFC.ZS',
          'SP.POP.DPND',
          'SP.POP.0014.TO.ZS',
          'SP.POP.65UP.TO.ZS',
          'SI.POV.DDAY',
          'GC.REV.SOCL.ZS',
          'SI.DST.FRST.10',
          'SL.UEM.TOTL.ZS']

wb_dic = ['gdp_grow',
              'savings_gdp',
              'educ_govexp_gdp',
              'legal_rights',
              'inflation',
              'subnutri_pop',
              'razao_dep',
              'sub14_pop',
              'idosos_pop',
              'pobreza_pop',
              'social_contrib',
              'inc_10',
              'unemployment']
              

# Dataframe que contem em uma coluna os codigos e outra os nomes
codigo_nome = pd.DataFrame([wb_cod, wb_dic]).T
codigo_nome.columns = ['wb_cod', 'wb_dic']


# Download das series do WB
pol_variables = wb.data.DataFrame(wb_cod)

# Copiando a série
pol_variables2 = pol_variables.copy()

# Tirando a média das linhas
pol_variables2['media'] = pol_variables2.mean(numeric_only=True, axis=1)
pol_variables2 = pol_variables2['media']

# Como tenho um multiindex, posso fazer o unstack e o indice de dentro vira as colunas
pol_variables2 = pol_variables2.unstack() 

# Renomeando as colunas usando o dicionario
d = codigo_nome.set_index('wb_cod').to_dict()
pol_variables2.columns = pol_variables2.columns.to_series().map(d['wb_dic'])

# Deletando ultima coluna
pol_variables2 = pol_variables2.drop(pol_variables2.columns[len(pol_variables2.columns)-1], axis=1)
# pol_variables2 = pol_variables2.drop(labels='wb_cod', axis=0)

# Criando uma coluna com nomes de países
pol_variables2['countrycode'] = pol_variables2.index

In [12]:
pol_variables2.tail()

series,inflation,social_contrib,legal_rights,gdp_grow,savings_gdp,educ_govexp_gdp,inc_10,pobreza_pop,unemployment,subnutri_pop,sub14_pop,idosos_pop,countrycode
economy,,,,,,,,,,,,,
XKX,1.695869,NaN,9.857143,5.164474,20.895186,NaN,3.641667,1.683333,NaN,NaN,NaN,NaN,XKX
YEM,17.495677,NaN,0.000000,1.869712,NaN,8.011537,3.066667,11.566667,11.655667,NaN,46.450087,2.999122,YEM
ZAF,7.903920,1.739414,5.000000,2.940889,20.973814,5.461569,1.057143,25.700000,27.999334,4.105556,36.994295,4.245798,ZAF
ZMB,38.223546,0.000377,8.714286,3.305475,34.478084,3.729464,1.344444,55.100000,13.080000,NaN,47.246728,2.413915,ZMB
ZWE,2.132165,1.301049,5.142857,2.910764,7.382431,9.576959,2.500000,31.600000,5.242000,NaN,45.453259,2.934481,ZWE


# 2) Construindo um dicionário

Nesta seção, irei construir um dicionário com **nomes dos países** (que por vezes estão escritos de diferentes maneiras) e **código dos países**. Assim, conseguirei manter um padrão (usando o código do país) entre as diferentes bases de dados

In [7]:
####################################################################
######################## --- Dicionário --- ########################
####################################################################

# Dicionario1: Extraindo os codigos dos países em PWT
countries_code = pwt[['country', 'countrycode']].drop_duplicates('country', keep = 'first')


# Consertando esse erro de label (muda nada na pratica, mas deixo padronizado ISO3)
ipd.loc[ipd['Country'] == 'Comoros', 'Code'] = 'COM' 
ipd.loc[ipd['Country'] == 'Eritrea', 'Code'] = 'ERI'


# Dicionario2: Extraindo os codigos dos países em IPD
countries_code2 = ipd[['Country', 'Code']].drop_duplicates('Country', keep = 'first')
countries_code2.columns = ['country', 'countrycode']


# Empilhando ambos os dicionarios
countries_code3 = pd.concat([countries_code, countries_code2], ignore_index=True).drop_duplicates('country', keep = 'first')

Empilhar o nome e os códigos dos países de dois dataframes (PWT e IPD) foi útil, já que podem existir grafias diferentes do país para um mesmo código. No entanto, isso não esgota os países, nem as grafias.

Por exemplo, existem países que estão no dataframe CBI e PWT que não estão nesse dicionário criado. Para resolver isso, abaixo eu adiciono manualmente os países que estão faltando.


Um potencial problema são países que foram fragmentados, por exemplo, Yuguslavia -> Serbia-Montenegro -> Serbia e Montenegro. Nesse caso especificamente eu optei por agregar YUG e Serbia-Montenegro durante o período que foi um único país, a partir da separação, separei em 2 codigos distindos. A grande questão é que a PWT (os dados de consumo) estão separados desde 1950 (não encontrei a metodologia para o split).

In [13]:
####################################################################


# Checando países em que o merge nao deu certo, isto é, existe o
# país em cbi2 (onde tem mais países) e não existe código corres
# pondente em fr2
# paises_sem_codigo = cbi2[cbi2['countrycode'].isnull()]['country']


paises_faltantes = np.array([
                            ['Antigua & Barbuda', 'ATG'],
                            ['Bosnia-Herzegovina', 'BIH'],
                            ['Cape Verde', 'CPV'],
                            ['Congo, Democratic Republic of / Za', 'COD'],
                            ['Congo, Republic of', 'COG'],
                            ['Ethiopia (incl. Eritrea)', 'ETH'],
                            ['Guinea Bissau', 'GNB'],
                            ['Hong Kong SAR', 'HKG'],
                            ['Iran', 'IRN'],
                            ['Ivory Coast', 'CIV'],
                            ['Korea, Republic of', 'KOR'],
                            ['Kosovo', 'XKX'], #Kosovo is not listed as an ISO standard country. The unofficial 2 and 3-digit codes are used by the European Commission and others until Kosovo is assigned an ISO code.
                            ['Laos', 'LAO'],
                            ['Myanmar (Burma)', 'MMR'],
                            ['Papua New Guinea', 'PNG'],
                            ['Russia', 'RUS'],
                            ['Samoa', 'WSM'],
                            ['San Marino', 'SMR'],
                            ['Serbia and Montenegro','YUG'], #2003, 2004, 2005 (data from CBI)
                            ['Solomon Islands', 'SLB'],
                            ['St. Kitts and Nevis', 'KNA'],
                            ['St. Lucia', 'LCA'],
                            ['Swaziland', 'SWZ'], # Mudou de nome para Eswatini (from CBI)
                            ['Syria', 'SYR'],
                            ['Timor-Leste', 'TLS'],
                            ['Tonga', 'TON'],
                            ['United States of America', 'USA'],
                            ['Vanuatu', 'VUT'],
                            ['Venezuela', 'VEN'],
                            ['Yemen, North', 'YEM'],
                            ['Yemen, North/Yemen Arab Rep.', 'YEM'],
                            ['Yugoslavia (Serbia-Montenegro)','YUG'], #92-2002 (serbia montenegro)
                            ['Yugoslavia (Socialist Rep)', 'YUG'], # 70-91 (socialist republic)
                            ['Zaire', 'COD'] # Rep. Demon. Congo entre 71-97
                            ])


# Transformando em dataframe do pandas
paises_faltantes = pd.DataFrame(paises_faltantes, columns = ['country', 'countrycode'])


# Empilhando ambos os dicionarios
countries_code3 = pd.concat([paises_faltantes, countries_code3], ignore_index=True)

# 3) Manipulando os dados

Nesta seção, para os dataframes que possuem séries temporais calculo a média total entre todos os anos, como uma forma de agregar as observações (*).

Para cada dataframe incorporo o dicionário de códigos, padronizando o nome de cada país para o seu respectivo código (ISO Alpha 3 Code).

Além disso, junto todos os dataframes.


---
(*)   Para agregar, posso testar usar a variância ou alguma outra métrica.



In [14]:
###########################################################################
######################## --- Manipulando Dados --- ########################
###########################################################################

#######
# CBI #
#######

# Renomeando coluna, preparando pro merge
cbi.rename(columns={'cname':'country'}, inplace=True)

# Calculando a média entre os anos, renomeando e acrescentando o código do país
cbi2 = pd.merge(cbi, countries_code3, on='country', how='left')
cbi2 = cbi2.groupby('countrycode').mean()
cbi2 = cbi2.rename_axis('countrycode').reset_index()

# Selecionando apenas as colunas que interessam
cbi2 = cbi2[['countrycode', 'lvau_garriga', 'lvaw_garriga']]


####################################################
# Verificando consistencia nos dados após agregação
####################################################

# Apenas deixo nomes de países unicos
teste = cbi.groupby('country').mean().rename_axis('country').reset_index()

# Possuo 191 nomes de países
print('Checando quantos nomes únicos de países tenho')
print(len(teste))
print()

# Selecionando variáveis que serão mais faceis de visualizar
teste = teste[['country', 'lvaw_garriga']]

# Left-Join com o dicionário. Assim, consigo ver os nomes dos países e os códigos (duplicados)
teste = pd.merge(teste, countries_code3, on='country', how='left')

# Verificando quais são os países duplicados. Em geral, sao países que mudaram de nome
dups = [x for x in teste['countrycode'].tolist() if teste['countrycode'].tolist().count(x) > 1]

# Países excluídos pq estão duplicados: 8
print('Checando quantos códigos de países tenho duplicados (país com mais de uma grafia)')
print(len(dups) - len( list(set(dups) ) ))
print()


print('Checando se a diferença no tamanho dos datasets sao devido aos países duplicados')
print(len(teste) - len(cbi2) == len(dups) - len( list(set(dups) ) ))

# Passa a ter 182 países
# Eu perco 8 por causa de mudança de nome: apenas deixo 1 de cada grupo abaixo


# Bosnia and Herzegovina
# Bosnia-Herzegovina


# Congo, Republic of
# Zaire


# Ethiopia
# Ethiopia (incl. Eritrea)


# Myanmar
# Myanmar (Burma)


# Serbia and Montenegro
# Yugoslavia (Serbia-Montenegro)
# Yugoslavia (Socialist Rep)

# Yemen
# Yemen, North
# Yemen, North/Yemen Arab Rep.


Checando quantos nomes únicos de países tenho
191

Checando quantos códigos de países tenho duplicados (país com mais de uma grafia)
8

Checando se a diferença no tamanho dos datasets sao devido aos países duplicados
True


In [15]:
#######
# PWT #
#######

# Renomeando coluna, preparando pro merge
fr.rename(columns={'Country':'country'}, inplace = True)

# - Alerta: Varias células indicam 'NAN' (não havia regra fiscal no momento). Poderia mudar para 0?
# Sim. Os papers que constroem indices de regras fiscais mudam para 0.
fr2  = fr[:]
fr2[['stab_n', 'stab_s']]  = fr2[['stab_n', 'stab_s']].fillna(0)


# Calculando a média entre os anos, renomeando e acrescentando o código do país
fr2  = pd.merge(fr2, countries_code3, on='country', how='left')
fr2  = fr2.groupby('countrycode').mean()
fr2  = fr2.rename_axis('countrycode').reset_index()


# Selecionando apenas as colunas que indicam regra fiscal que estabiliza ou nao o ciclo
fr2[['stab_n', 'stab_s']] = fr2[['stab_n', 'stab_s']].fillna(0)
fr2  = fr2[['countrycode', 'stab_n', 'stab_s']]


####################################################
# Verificando consistencia nos dados após agregação
####################################################

# Apenas deixo nomes de países unicos
teste  = fr[:]
teste[['stab_n', 'stab_s']]  = teste[['stab_n', 'stab_s']].fillna(0)
teste = teste.groupby('country').mean().rename_axis('country').reset_index()

# Possuo 76 nomes de países
print('Checando quantos nomes únicos de países tenho')
print(len(teste))
print()

# Selecionando variáveis que serão mais faceis de visualizar
teste = teste[['country', 'stab_n']]

# Fazendo o left-join
teste = pd.merge(teste, countries_code3, on='country', how='left')

# Verificando quais são os países duplicados. Em geral, sao países que mudaram de nome
dups = [x for x in teste['countrycode'].tolist() if teste['countrycode'].tolist().count(x) > 1]

# Países excluídos pq estão duplicados: 0
print('Checando quantos códigos de países tenho duplicados (país com mais de uma grafia)')
print(len(dups) - len( list(set(dups) ) ))
print()


# Verificando se a diferença no tamanho dos datasets sao devido aos países duplicados
print('Checando se a diferença no tamanho dos datasets sao devido aos países duplicados')
print(len(teste) - len(fr2) == len(dups) - len( list(set(dups) ) ))

Checando quantos nomes únicos de países tenho
96

Checando quantos códigos de países tenho duplicados (país com mais de uma grafia)
0

Checando se a diferença no tamanho dos datasets sao devido aos países duplicados
True


In [16]:
# Calculando a média entre os anos e renomeando
pwt['gdp_percapta'] = pwt['csh_c']/pwt['pop']
pwt['emp_pop'] = pwt['emp']/pwt['pop']
pwt2 = pwt.groupby('countrycode').mean()
pwt2 = pwt2.rename_axis('countrycode').reset_index()

# Selecionando pib per capita, PO, horas trabalhadas média por pessoa, Human capital index
# Welfare-relevant TFP, Share of labour compensation in GDP
pwt2 = pwt2[['countrycode', 'gdp_percapta', 'emp_pop', 'avh', 'hc', 'rwtfpna', 'labsh']]


################################################################

# Renomenado a coluna Code
ipd2 = ipd[:]
ipd2.rename(columns={'Code':'countrycode'}, inplace=True)
ipd2 = ipd2.drop(['Country', 'Year', 'Income level', 'Region'], axis = 1)

################################################################

# Juntando todas as variáveis em um único dataframe
dataframe = pd.merge( cbi2, fr2, on ='countrycode', how='outer')
dataframe = pd.merge( dataframe, pwt2, on='countrycode', how='outer')
dataframe = pd.merge( dataframe, ipd2, on='countrycode', how='outer')
dataframe = pd.merge( dataframe, pol_variables2, on='countrycode', how='outer')

# Os países que nao tem FR nao aparecem nesse banco de dados. Isto é, as regras  = 0
dataframe[['stab_n', 'stab_s']] = dataframe[['stab_n', 'stab_s']].fillna(0)

dataframe[['countrycode', 'gdp_percapta']].head( n = 10)

,countrycode,gdp_percapta
0,AFG,NaN
1,AGO,0.031580
2,ALB,0.256826
3,ARE,0.161813
4,ARG,0.025891
5,ARM,0.244836
6,ATG,10.170944
7,AUS,0.040376
8,AUT,0.079310
9,AZE,0.058354


In [17]:
###############################################################

# Verificando quais são os países duplicados. Em geral, sao países que mudaram de nome
dups = [x for x in dataframe['countrycode'].tolist() if dataframe['countrycode'].tolist().count(x) > 1]

# Total de países no meu dataframe final: 198
paises1 = dataframe.drop_duplicates('countrycode', keep = 'first')
print('Checando quantos países únicos tenho na base final')
print(len(paises1))
print()

# Zero países duplicados, o que é bom indicio
print('Checando quantos duplicados')
print(len(dups))
print('Checando quais duplicados')
print(dups)
print()

# Verificando o total de países: 198
paises2 = countries_code3.drop_duplicates('countrycode', keep = 'first')
print('Total de países no dicionário')
print(len(paises2))
print()

print('Todos os países estão no dicionário?')
print(len(paises1)) == print(len(paises2))

Checando quantos países únicos tenho na base final
268

Checando quantos duplicados
0
Checando quais duplicados
[]

Total de países no dicionário
198

Todos os países estão no dicionário?
268
198


True

#4) Exportando 

In [19]:
# Exportanto dados

dataframe.to_csv('countries_data.csv')
files.download('countries_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>